# Data Cleaning & Feature Engineering

**Objective:** Transform raw marketing data into analysis-ready features for channel performance analysis, customer segmentation, and attribution modeling.

**This Notebook:**
- Load cleaned datasets from outputs/
- Engineer marketing metrics (CTR, CVR, ROAS, CAC)
- Create time-based features (day of week, month, seasonality)
- Calculate customer lifetime value (LTV)
- Identify and handle outliers
- Create analysis-ready datasets

**Previous Notebook:** 01_data_acquisition.ipynb  
**Next Notebook:** 03_exploratory_analysis.ipynb

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


## 1. Load Cleaned Datasets

Load the cleaned data exported from notebook 01.

In [2]:
# Load datasets from outputs folder
print("Loading datasets...")

campaigns_df = pd.read_csv('../outputs/campaigns_clean.csv')
daily_performance_df = pd.read_csv('../outputs/daily_performance_clean.csv', parse_dates=['date'])
customers_df = pd.read_csv('../outputs/customers_clean.csv', parse_dates=['acquisition_date'])
transactions_df = pd.read_csv('../outputs/transactions_clean.csv', parse_dates=['transaction_date'])
performance_enriched = pd.read_csv('../outputs/performance_enriched.csv', parse_dates=['date'])

print(f"Campaigns: {len(campaigns_df):,} records")
print(f"Daily Performance: {len(daily_performance_df):,} records")
print(f"Customers: {len(customers_df):,} records")
print(f"Transactions: {len(transactions_df):,} records")
print(f"Performance Enriched: {len(performance_enriched):,} records")

Loading datasets...
Campaigns: 25 records
Daily Performance: 1,000 records
Customers: 1,000 records
Transactions: 1,000 records
Performance Enriched: 1,000 records


## 2. Feature Engineering: Marketing Metrics

Calculate key performance metrics:
- **CTR (Click-Through Rate)**: Clicks / Impressions
- **CVR (Conversion Rate)**: Conversions / Clicks  
- **CPC (Cost Per Click)**: Spend / Clicks
- **ROAS (Return on Ad Spend)**: Revenue / Spend
- **CAC (Customer Acquisition Cost)**: Spend / Conversions

In [3]:
# Create a copy to work with
perf_features = performance_enriched.copy()

# Calculate marketing metrics
perf_features['ctr'] = (perf_features['clicks'] / perf_features['impressions'].replace(0, np.nan)) * 100
perf_features['cvr'] = (perf_features['conversions'] / perf_features['clicks'].replace(0, np.nan)) * 100
perf_features['cpc'] = perf_features['spend'] / perf_features['clicks'].replace(0, np.nan)
perf_features['roas'] = perf_features['revenue'] / perf_features['spend'].replace(0, np.nan)
perf_features['cac'] = perf_features['spend'] / perf_features['conversions'].replace(0, np.nan)

# Profit metrics
perf_features['profit'] = perf_features['revenue'] - perf_features['spend']
perf_features['profit_margin'] = (perf_features['profit'] / perf_features['revenue'].replace(0, np.nan)) * 100

print("Marketing Metrics Summary:")
print(perf_features[['ctr', 'cvr', 'cpc', 'roas', 'cac', 'profit_margin']].describe())

Marketing Metrics Summary:
          ctr     cvr     cpc    roas    cac  profit_margin
count 1000.00 1000.00 1000.00 1000.00 969.00         969.00
mean     2.44    1.94    1.34    6.45  91.04         -29.40
std      1.01    1.09    0.99   11.52  70.37         102.04
min      0.84    0.00    0.08    0.00   1.69        -544.78
25%      1.66    1.09    0.64    0.53  51.05         -81.93
50%      2.18    1.70    1.15    0.83  82.46         -16.44
75%      3.06    2.76    2.09    1.37 125.59          30.43
max      5.81    4.55    4.36   50.04 340.12          98.00


## 3. Time-Based Features

Extract temporal patterns for seasonality analysis.

In [4]:
# Extract time-based features
perf_features['year'] = perf_features['date'].dt.year
perf_features['month'] = perf_features['date'].dt.month
perf_features['month_name'] = perf_features['date'].dt.month_name()
perf_features['week'] = perf_features['date'].dt.isocalendar().week
perf_features['day_of_week'] = perf_features['date'].dt.dayofweek
perf_features['day_name'] = perf_features['date'].dt.day_name()
perf_features['quarter'] = perf_features['date'].dt.quarter
perf_features['is_weekend'] = perf_features['day_of_week'].isin([5, 6]).astype(int)

# Season (Northern Hemisphere)
perf_features['season'] = perf_features['month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})

print("Time Features Added:")
print(perf_features[['date', 'day_name', 'month_name', 'quarter', 'season', 'is_weekend']].head())

Time Features Added:
        date   day_name month_name  quarter  season  is_weekend
0 2024-01-13   Saturday    January        1  Winter           1
1 2024-01-14     Sunday    January        1  Winter           1
2 2024-01-15     Monday    January        1  Winter           0
3 2024-01-16    Tuesday    January        1  Winter           0
4 2024-01-17  Wednesday    January        1  Winter           0


## 4. Customer Features: Lifetime Value (LTV)

Calculate customer-level metrics including total LTV, purchase frequency, and recency.

In [5]:
# Calculate customer LTV and behavior metrics
customer_features = customers_df.copy()

# Aggregate transaction data per customer
customer_transactions = transactions_df.groupby('customer_id').agg({
    'order_value': ['sum', 'mean', 'count'],
    'transaction_date': ['min', 'max'],
    'products_purchased': 'sum',
    'discount_applied': 'sum'
}).reset_index()

# Flatten column names
customer_transactions.columns = ['customer_id', 'total_ltv', 'avg_order_value', 
                                 'num_orders', 'first_purchase_date', 
                                 'last_purchase_date', 'total_products', 'total_discounts']

# Merge with customer data
customer_features = customer_features.merge(customer_transactions, on='customer_id', how='left')

# Fill NaN for customers with only first purchase
customer_features['total_ltv'] = customer_features['total_ltv'].fillna(customer_features['first_order_value'])
customer_features['num_orders'] = customer_features['num_orders'].fillna(1)
customer_features['avg_order_value'] = customer_features['avg_order_value'].fillna(customer_features['first_order_value'])

# Calculate additional metrics
customer_features['days_as_customer'] = (
    customer_features['last_purchase_date'].fillna(customer_features['acquisition_date']) - 
    customer_features['acquisition_date']
).dt.days

customer_features['is_repeat_customer'] = (customer_features['num_orders'] > 1).astype(int)

print("Customer Features Summary:")
print(customer_features[['total_ltv', 'num_orders', 'avg_order_value', 'days_as_customer']].describe())

Customer Features Summary:
       total_ltv  num_orders  avg_order_value  days_as_customer
count    1000.00     1000.00          1000.00           1000.00
mean      121.48        1.57            75.83             19.20
std       112.81        1.14            24.43             39.71
min        32.81        1.00            27.89              0.00
25%        63.53        1.00            55.66              0.00
50%        90.03        1.00            73.51              0.00
75%       126.08        2.00            93.31             25.00
max      1006.88        9.00           141.81            288.00


## 5. Channel-Level Aggregations

Create channel performance summaries for comparison analysis.

In [6]:
# Aggregate metrics by channel
channel_summary = perf_features.groupby('channel').agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'clicks': 'sum',
    'impressions': 'sum',
    'roas': 'mean',
    'cac': 'mean',
    'ctr': 'mean',
    'cvr': 'mean',
    'profit': 'sum',
    'profit_margin': 'mean'
}).round(2)

# Calculate overall metrics
channel_summary['overall_roas'] = channel_summary['revenue'] / channel_summary['spend']
channel_summary['overall_cac'] = channel_summary['spend'] / channel_summary['conversions']

# Sort by ROAS
channel_summary = channel_summary.sort_values('overall_roas', ascending=False)

print("Channel Performance Summary:")
print(channel_summary)

Channel Performance Summary:
                spend    revenue  conversions  clicks  impressions  roas  \
channel                                                                    
email        40666.32 1110689.26        14919  412107     16266386 25.99   
affiliate    12874.66   11620.29          126    4462        95346  0.85   
paid_search  91256.29   76121.99          889   37952      1042803  0.76   
social      117419.01   95330.19         1443  101496      5435875  0.70   
display      75589.41   45038.98          822  103897      8398755  0.55   

               cac  ctr  cvr     profit  profit_margin  overall_roas  \
channel                                                                
email         3.28 2.47 3.41 1070022.94          95.56         27.31   
affiliate   133.62 4.55 2.62   -1254.37         -51.15          0.90   
paid_search 133.51 3.53 2.11  -15134.30         -57.63          0.83   
social      104.08 1.80 1.21  -22088.82         -58.32          0.81   
displa

## 6. Outlier Detection

Identify and flag outliers using IQR method for key metrics.

In [7]:
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    return outliers, lower_bound, upper_bound

# Check for outliers in key metrics
metrics_to_check = ['spend', 'revenue', 'roas', 'cac', 'ctr', 'cvr']

print("Outlier Detection Results:")
print("=" * 60)

for metric in metrics_to_check:
    if metric in perf_features.columns:
        outliers, lower, upper = detect_outliers_iqr(perf_features[perf_features[metric].notna()], metric)
        pct_outliers = (outliers.sum() / len(outliers)) * 100
        
        print(f"\n{metric.upper()}:")
        print(f"  Outliers: {outliers.sum()} ({pct_outliers:.1f}%)")
        print(f"  Range: [{lower:.2f}, {upper:.2f}]")
        
        # Flag outliers in dataframe
        perf_features[f'{metric}_is_outlier'] = False
        perf_features.loc[perf_features.index.isin(outliers[outliers].index), f'{metric}_is_outlier'] = True

Outlier Detection Results:

SPEND:
  Outliers: 80 (8.0%)
  Range: [-128.27, 718.04]

REVENUE:
  Outliers: 185 (18.5%)
  Range: [-976.33, 2010.66]

ROAS:
  Outliers: 227 (22.7%)
  Range: [-0.73, 2.62]

CAC:
  Outliers: 35 (3.6%)
  Range: [-60.78, 237.42]

CTR:
  Outliers: 10 (1.0%)
  Range: [-0.45, 5.17]

CVR:
  Outliers: 0 (0.0%)
  Range: [-1.43, 5.28]


## 7. Campaign-Level Features

Aggregate daily performance to campaign level for high-level analysis.

In [8]:
# Create campaign-level summary
campaign_performance = perf_features.groupby(['campaign_id', 'campaign_name', 'channel']).agg({
    'spend': 'sum',
    'revenue': 'sum',
    'conversions': 'sum',
    'clicks': 'sum',
    'impressions': 'sum',
    'date': ['min', 'max']
}).reset_index()

# Flatten columns
campaign_performance.columns = ['campaign_id', 'campaign_name', 'channel', 'total_spend', 
                               'total_revenue', 'total_conversions', 'total_clicks', 
                               'total_impressions', 'start_date', 'end_date']

# Calculate campaign-level metrics
campaign_performance['campaign_roas'] = campaign_performance['total_revenue'] / campaign_performance['total_spend']
campaign_performance['campaign_cac'] = campaign_performance['total_spend'] / campaign_performance['total_conversions']
campaign_performance['campaign_ctr'] = (campaign_performance['total_clicks'] / campaign_performance['total_impressions']) * 100
campaign_performance['campaign_cvr'] = (campaign_performance['total_conversions'] / campaign_performance['total_clicks']) * 100
campaign_performance['campaign_duration_days'] = (campaign_performance['end_date'] - campaign_performance['start_date']).dt.days + 1
campaign_performance['profit'] = campaign_performance['total_revenue'] - campaign_performance['total_spend']

# Add efficiency score (normalized ROAS * normalized conversion rate)
campaign_performance['efficiency_score'] = (
    (campaign_performance['campaign_roas'] / campaign_performance['campaign_roas'].max()) * 0.6 +
    (campaign_performance['campaign_cvr'] / campaign_performance['campaign_cvr'].max()) * 0.4
) * 100

print("Campaign Performance Summary:")
print(campaign_performance.sort_values('campaign_roas', ascending=False).head(10))

Campaign Performance Summary:
    campaign_id            campaign_name      channel  total_spend  \
9            10        Email Campaign 10        email      6013.21   
12           13        Email Campaign 13        email      9569.22   
7             8         Email Campaign 8        email     12024.91   
15           16        Email Campaign 16        email     13058.98   
4             5     Affiliate Campaign 5    affiliate     12874.66   
14           15  Paid Search Campaign 15  paid_search     36738.54   
11           12       Social Campaign 12       social     32827.11   
5             6   Paid Search Campaign 6  paid_search     25309.65   
1             2        Social Campaign 2       social     29417.75   
16           17       Social Campaign 17       social     16237.73   

    total_revenue  total_conversions  total_clicks  total_impressions  \
9       174227.46               2323         62807            2405237   
12      265163.46               3478         96688   

## 8. Export Engineered Datasets

Save feature-engineered datasets for analysis notebooks.

In [9]:
# Export engineered datasets
print("Exporting feature-engineered datasets...")

perf_features.to_csv('../outputs/performance_features.csv', index=False)
print("Saved: performance_features.csv")

customer_features.to_csv('../outputs/customer_features.csv', index=False)
print("Saved: customer_features.csv")

channel_summary.to_csv('../outputs/channel_summary.csv')
print("Saved: channel_summary.csv")

campaign_performance.to_csv('../outputs/campaign_performance.csv', index=False)
print("Saved: campaign_performance.csv")

print("\nFeature engineering complete!")

Exporting feature-engineered datasets...
Saved: performance_features.csv
Saved: customer_features.csv
Saved: channel_summary.csv
Saved: campaign_performance.csv

Feature engineering complete!


## Summary

**Features Created:**
- ✅ Marketing metrics: CTR, CVR, CPC, ROAS, CAC, profit margin
- ✅ Time features: day of week, month, quarter, season, weekend flag
- ✅ Customer LTV: total lifetime value, purchase frequency, recency
- ✅ Channel aggregations: performance by marketing channel
- ✅ Campaign summaries: campaign-level performance metrics
- ✅ Outlier flags: IQR-based outlier detection

**Datasets Exported:**
1. `performance_features.csv` - Daily metrics with all engineered features
2. `customer_features.csv` - Customer-level LTV and behavior metrics
3. `channel_summary.csv` - Channel performance aggregations
4. `campaign_performance.csv` - Campaign-level summaries

**Next Steps:**
- Notebook 03: Exploratory Data Analysis (EDA)
- Visualize distributions, correlations, and trends
- Identify insights for channel optimization